In [1]:
!pip install datasets transformers

     -------------------------------------- 452.9/452.9 kB 3.5 MB/s eta 0:00:00
  Using cached fsspec-2022.11.0-py3-none-any.whl (139 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
     ---------------------------------------- 132.9/132.9 kB ? eta 0:00:00
     ------------------------------------- 323.5/323.5 kB 19.6 MB/s eta 0:00:00
     --------------------------------------- 20.3/20.3 MB 20.4 MB/s eta 0:00:00
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached charset_normalizer-2.1.1-py3-none-any.whl (39 kB)
     ---------------------------------------- 56.8/56.8 kB 2.9 MB/s eta 0:00:00


In [2]:
import transformers

print(transformers.__version__)

4.24.0


### Comparison to Example Script

In [3]:
squad_v2 = False
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

In [4]:
from datasets import load_dataset, load_metric
datasets = load_dataset("squad_v2" if squad_v2 else "squad")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to C:/Users/lenna/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [6]:
datasets['train'][0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

### Our labeled Questions

In [76]:
import json
import pandas as pd
f = open('./generated_training_data/qa_russel_sage.json') # Exported file from LabelStudio
data = json.load(f)
data.append(open('./generated_training_data/miele_qa.json'))
data.append(open('./generated_training_data/melitta_qa.json'))
data.append(open('./generated_training_data/qa_complete_marius.json'))
data.append(open('./generated_training_data/qa_Jochen.json'))
print(len(data))

In [77]:
data[5]["answer"]

[{'start': 856,
  'end': 978,
  'text': 'Use the appliance according to these instructions. Any misuse may cause potential injury, electric shock or other hazards.',
  'labels': ['Answer']}]

In [78]:
data[5]['answer'][0]['text']

'Use the appliance according to these instructions. Any misuse may cause potential injury, electric shock or other hazards.'

In [79]:
data[0]['answer'][0]['start']

0

Bring the questions in the same format described in the muster example before

In [80]:
questions_right_format = []

for i, q in enumerate(data):
    if "answer" in q:
      question_dic = {}

      question_dic['id'] = q['id']
      question_dic['title'] = f"Test_{i}"
      question_dic['context'] = q['text']
      question_dic['question'] = q['question']

      answer_dic = {}
      answer_dic['text'] = [q['answer'][0]['text']]
      answer_dic['answer_start'] = [int(q['answer'][0]['start'])]

      question_dic['answers'] = answer_dic

      questions_right_format.append(question_dic)


UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 6191: character maps to <undefined>

In [29]:
qa_df = pd.DataFrame(questions_right_format)

In [30]:
qa_df['title'] = "Test"
qa_df.rename({'text': 'context', 'answer': 'answers'}, inplace=True)

In [31]:
qa_df.iloc[0]['context'], qa_df.iloc[0]['question'], type(qa_df.iloc[0]['answers']['answer_start'][0])

('Follow basic safety precautions, including: This appliance can be used by children aged from 8 years and above and persons with reduced physical, sensory or mental capabilities or lack of experience and knowledge if they have been supervised/instructed and understand the hazards involved. Children shall not play with the appliance. Cleaning and user maintenance shall not be done by children unless they are older than 8 and supervised. Keep the appliance and cable out of reach of children under 8 years. Misuse of the appliance may cause injury. h The surfaces of the appliance will get hot. Residual heat will keep the surfaces hot after use. If the cable is damaged, it must be replaced by the manufacturer, its service agent, or someone similarly qualified, to avoid hazard. The surface of the heating element is subject to residual heat after use. Use the appliance according to these instructions. Any misuse may cause potential injury, electric shock or other hazards. This appliance can 

In [32]:
qa_df

,id,title,context,question,answers
0,1,Test,"Follow basic safety precautions, including: Th...",Can the machine used by children?,"{'text': ['Follow basic safety precautions, in..."
1,2,Test,"Follow basic safety precautions, including: Th...",is the machine safe for children?,{'text': ['This appliance can be used by child...
2,4,Test,"Follow basic safety precautions, including: Th...",What do I have to do if the cable is damaged?,{'text': ['Donï¿½t use the appliance if itï¿½s...
3,6,Test,"Follow basic safety precautions, including: Th...",is the machine water resistant?,{'text': ['Do not use this appliance near bath...
4,9,Test,"Follow basic safety precautions, including: Th...",Can the machine overheat?,{'text': ['The surfaces of the appliance will ...
...,...,...,...,...,...
149,156,Test,OPTIMUM 25-35 SEC BALANCED GRIND SHOT TIME TOO...,Why does my coffee taste sour?,{'text': ['ASTRINGENT TOO COARSE UNDER 20 SEC ...
150,157,Test,Press MENU button until SHOT TEMP icon flashe...,How can I change the termperature?What tempera...,{'text': ['Press MENU button until SHOT TEMP i...
151,158,Test,Press MENU button and the current Auto Start ...,How can I set the auto start?,{'text': ['Press MENU button and the current A...
152,159,Test,The machine will detect when 200 extractions h...,When do I have to clean the machine?,{'text': ['The machine will detect when 200 ex...


Create Dataset Objects for Train/Test split (could very well be more cleverly done, but I dont care)

In [33]:
# shuffle the data
qa_df = qa_df.sample(frac=1)
# take 80% as training samples
no_train_samples = int(len(qa_df) * 0.8)

qa_df_train = qa_df.iloc[0:no_train_samples]
qa_df_test = qa_df.iloc[no_train_samples:-1]

In [34]:
qa_dic_train = qa_df_train.to_dict(orient='records')
qa_dic_test = qa_df_test.to_dict(orient='records')

In [35]:
qa_dic_train[0]

{'id': 1,
 'title': 'Test',
 'context': 'Follow basic safety precautions, including: This appliance can be used by children aged from 8 years and above and persons with reduced physical, sensory or mental capabilities or lack of experience and knowledge if they have been supervised/instructed and understand the hazards involved. Children shall not play with the appliance. Cleaning and user maintenance shall not be done by children unless they are older than 8 and supervised. Keep the appliance and cable out of reach of children under 8 years. Misuse of the appliance may cause injury. h The surfaces of the appliance will get hot. Residual heat will keep the surfaces hot after use. If the cable is damaged, it must be replaced by the manufacturer, its service agent, or someone similarly qualified, to avoid hazard. The surface of the heating element is subject to residual heat after use. Use the appliance according to these instructions. Any misuse may cause potential injury, electric shoc

In [36]:
from datasets import Dataset
from datasets import DatasetDict
dataset_train = Dataset.from_pandas(qa_df_train)
dataset_test = Dataset.from_pandas(qa_df_test)
dataset_train = dataset_train.remove_columns(["__index_level_0__"])
dataset_test = dataset_test.remove_columns(["__index_level_0__"])

In [37]:
qa_dataset = DatasetDict({
    "train": dataset_train,
    "test": dataset_test
})

In [38]:
qa_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 120
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 33
    })
})

In [39]:
qa_dataset['train'][0]

{'id': 1,
 'title': 'Test',
 'context': 'Follow basic safety precautions, including: This appliance can be used by children aged from 8 years and above and persons with reduced physical, sensory or mental capabilities or lack of experience and knowledge if they have been supervised/instructed and understand the hazards involved. Children shall not play with the appliance. Cleaning and user maintenance shall not be done by children unless they are older than 8 and supervised. Keep the appliance and cable out of reach of children under 8 years. Misuse of the appliance may cause injury. h The surfaces of the appliance will get hot. Residual heat will keep the surfaces hot after use. If the cable is damaged, it must be replaced by the manufacturer, its service agent, or someone similarly qualified, to avoid hazard. The surface of the heating element is subject to residual heat after use. Use the appliance according to these instructions. Any misuse may cause potential injury, electric shoc

### Prepare the Training Stuff

In [40]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

C:\Python39\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\lenna\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [41]:
pad_on_right = tokenizer.padding_side == "right"

In [42]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [43]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [44]:
features = prepare_train_features(qa_dataset['train'][:5])

In [45]:
tokenized_datasets = qa_dataset.map(prepare_train_features, batched=True, remove_columns=qa_dataset["train"].column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

### Training

In [46]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this mode

In [47]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-coffee",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20,
    weight_decay=0.01,
    logging_strategy="epoch"
)

In [48]:
from transformers import default_data_collator

data_collator = default_data_collator

In [49]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

C:\Python39\lib\site-packages\setuptools\distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


In [50]:
trainer.train()

C:\Python39\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 167
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 220
  Number of trainable parameters = 66364418


Epoch,Training Loss,Validation Loss


***** Running Evaluation *****
  Num examples = 40
  Batch size = 16
***** Running Evaluation *****
  Num examples = 40
  Batch size = 16
***** Running Evaluation *****
  Num examples = 40
  Batch size = 16
***** Running Evaluation *****
  Num examples = 40
  Batch size = 16
***** Running Evaluation *****
  Num examples = 40
  Batch size = 16
***** Running Evaluation *****
  Num examples = 40
  Batch size = 16
***** Running Evaluation *****
  Num examples = 40
  Batch size = 16
***** Running Evaluation *****
  Num examples = 40
  Batch size = 16
***** Running Evaluation *****
  Num examples = 40
  Batch size = 16
***** Running Evaluation *****
  Num examples = 40
  Batch size = 16
***** Running Evaluation *****
  Num examples = 40
  Batch size = 16
***** Running Evaluation *****
  Num examples = 40
  Batch size = 16
***** Running Evaluation *****
  Num examples = 40
  Batch size = 16
***** Running Evaluation *****
  Num examples = 40
  Batch size = 16
***** Running Evaluation *****
  N

TrainOutput(global_step=220, training_loss=2.5435027555985883, metrics={'train_runtime': 3765.8721, 'train_samples_per_second': 0.887, 'train_steps_per_second': 0.058, 'total_flos': 327286015580160.0, 'train_loss': 2.5435027555985883, 'epoch': 20.0})

In [51]:
trainer.save_model("test-coffee-trained")

Saving model checkpoint to test-coffee-trained
Configuration saved in test-coffee-trained\config.json
Model weights saved in test-coffee-trained\pytorch_model.bin
tokenizer config file saved in test-coffee-trained\tokenizer_config.json
Special tokens file saved in test-coffee-trained\special_tokens_map.json


### Evaluation

In [52]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

In [53]:
output.start_logits.shape, output.end_logits.shape

(torch.Size([16, 384]), torch.Size([16, 384]))

In [54]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

(tensor([13,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 14,  0, 12,  0]),
 tensor([33,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 33]))

In [55]:
n_best_size = 20

In [56]:
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )

In [57]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [58]:
test_features = qa_dataset["test"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=qa_dataset["test"].column_names
)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [59]:
raw_predictions = trainer.predict(test_features)

The following columns in the test set don't have a corresponding argument in `DistilBertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping. If example_id, offset_mapping are not expected by `DistilBertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 40
  Batch size = 16


In [60]:
test_features.set_format(type=test_features.format["type"], columns=list(test_features.features.keys()))

In [61]:
max_answer_length = 30

In [72]:
example = qa_dataset["test"][0]

In [73]:
start_logits = output.start_logits[2].cpu().numpy()
end_logits = output.end_logits[2].cpu().numpy()
offset_mapping = test_features[2]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = example["context"]
question = example["question"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
print("Context: \n" + context)
print("Question: \n" + question)
for valid_answer in valid_answers:
    print(valid_answer)

Context: 
â€¢ The appliance and its cord should be kept out of reach of children aged 8 years and younger. â€¢ Do not use attachments other than those provided with the appliances. â€¢ Do not attempt to operate the appliance by any method other than those described in this booklet. â€¢ Do not move the appliance whilst in operation. â€¢ Do not use any other liquid apart from cold mains/town water. We do not recommend the use of highly filtered, de- mineralised or distilled water as this may affect the taste of the coffee and how the espresso machine is designed to function. â€¢ Never use the appliance without water in the water tank. â€¢ Ensure the portafilter is firmly inserted and secured into the brewing head before using the machine. â€¢ Never remove the portafilter during the brewing operation as the machine is under pressure. â€¢ Do not place anything, other than cups for warming, on top of the appliance. â€¢ Use caution when operating machine as metal surfaces are liable to get h

In [75]:
example["answers"]

{'answer_start': [1804],
 'text': ['Do not operate the appliance if it is in an enclosed space, or within a cupboard.']}

In [64]:
import collections

examples = qa_dataset["test"]
features = test_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [65]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

In [66]:
final_predictions = postprocess_qa_predictions(qa_dataset["test"], test_features, raw_predictions.predictions)

Post-processing 33 example predictions split into 40 features.


  0%|          | 0/33 [00:00<?, ?it/s]

In [67]:
metric = load_metric("squad_v2" if squad_v2 else "squad")

C:\Users\lenna\AppData\Local\Temp/ipykernel_22280/3256360914.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad_v2" if squad_v2 else "squad")


In [68]:
formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]

In [69]:
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in qa_dataset["test"]]
metric.compute(predictions=formatted_predictions, references=references)

{'exact_match': 0.0, 'f1': 5.596921763904686}